In [345]:
import pandas as pd
import numpy as np

In [ ]:
import re

# Load the Excel files
address_df = pd.read_excel('Book2_11dec.xlsm')
street_mapping_df = pd.read_excel('Copy of Dictionary master (1).xlsx', sheet_name='Street Names')
artery_mapping_df = pd.read_excel('Copy of Dictionary master (1).xlsx', sheet_name='Art')

# Initialize an empty list to store processed data
processed_data = []

# Iterate through each row of the address DataFrame
for index, row in address_df.iterrows():
    address = str(row['COMBO'])
    wr = str(row['ID'])

    # Check if the address should be skipped
    if pd.isna(address) or "SKIP" in str(address):
        continue  # Skip this address and move to the next iteration

    # Extract house number, compass, street name, and artery name
    #house_number_match = re.search(r'^[\d-]+', address)
    house_number_match = re.search(r'^[A-Z\d-]+', address)
    house_number = house_number_match.group() if house_number_match is not None else ''

    # Extract compass point if present in the address
    compass_point_match = re.search(r'(?<!\w)([NESW]|North|East|South|West)(?!\w)', address, re.IGNORECASE)
    compass_point = compass_point_match.group() if compass_point_match else ''

    # Split the address into words
    address_words = address.split()

    # Extract the last word as the artery name
    artery_name = address_words[-1] if address_words else ''

    # Remove the last word from the address_words list
    address_words = address_words[:-1]

    # # Extract street name from the remaining words
    # street_name = ' '.join(address_words)

    # Extract street name from the remaining words without house number and compass
    street_name = ' '.join([word for word in address_words if word not in [house_number, compass_point]])

    # Include first letter of street name if it's the same as compass
    if compass_point and street_name.lower().startswith(compass_point.lower()):
        street_name = street_name[len(compass_point):].strip()

    # If compass is present, try to capture the street name after compass and before artery
    if compass_point and not street_name:
        street_name_match = re.search(rf'{compass_point}\s*(\w+)\s+[A-Z]+', address, re.IGNORECASE)
        street_name = street_name_match.group(1) if street_name_match else ''

    # Find replacement for street name using mapping
    street_mapping = street_mapping_df.set_index('StreetName')['Standard Notation'].to_dict()
    if street_name.upper() in street_mapping:
        street_name = street_mapping[street_name.upper()]

    artery_mapping = artery_mapping_df.set_index('StreetNamePostType')['Standard Notation'].to_dict()
    if artery_name.upper() in artery_mapping:
        artery_name = artery_mapping[artery_name.upper()]

    # Append the processed data to the list
    processed_data.append([wr, house_number, compass_point, street_name, artery_name, address])

# Create a new DataFrame with processed data
processed_df = pd.DataFrame(processed_data, columns=['ID','House Number', 'Compass', 'Street Name', 'Artery Name', 'ADDRESS'])

# Write the processed DataFrame to a new Excel file
processed_df.to_excel('BOOK2_11DEC_SEPARATED.xlsx', index=False)

In [346]:
first_df = pd.read_excel('YEAR TO DATE INESGTION.xlsx')

In [347]:
first_df.iloc[:,1:] = first_df.iloc[:,1:].applymap(lambda x:x.upper() if isinstance(x,str) else x)

In [348]:
mapping_df = pd.read_excel('mapping2.xlsx')

In [349]:
first_df['MeterLocationAtmosphere'] = first_df['MeterLocationAtmosphere'].str.split(';').str[0]

In [350]:
first_df['MeterLocationAtmosphere'] = first_df['MeterLocationAtmosphere'].str.split(',').str[0]

In [351]:
mapping_dict = mapping_df.set_index('normalized name')['column name'].to_dict()

In [352]:
default_values_dict = mapping_df.set_index('normalized name')['default value'].apply(lambda x: x.split(',') if isinstance(x, str) else x).to_dict()

In [353]:
new_df = pd.read_excel('SLI_Template.xlsx')

In [354]:
new_df = pd.DataFrame(columns = new_df.columns)

In [355]:
mapping_dict = mapping_df.set_index('normalized name')['column name'].to_dict()

In [314]:
#default_values_dict = mapping_df.set_index('normalized name')['default value'].apply(lambda x: x.split(',') if isinstance(x, str) else x).to_dict()

In [315]:
#new_df = pd.DataFrame(columns = new_df.columns)

In [356]:
for column in new_df.columns:
    if column in mapping_dict and mapping_dict[column] in first_df.columns:
        column_name = mapping_dict[column]
        new_df[column] = first_df[column_name]
        #new_df[column] = first_df[column_name].where(first_df['Is the building accessible?']!='NO')
    else:
        new_df[column]=new_df[column]

In [357]:
new_df['BuildingAccessible'] = new_df['BuildingAccessible'].fillna('YES')
new_df['OutsideSet'] = new_df['OutsideSet'].fillna('NO')
new_df['PressureRegulatorInBuilding'] = new_df['PressureRegulatorInBuilding'].fillna('YES')
new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].fillna('SINGLE')
new_df['MeterLocation'] = new_df['MeterLocation'].fillna('BASEMENT')
new_df['MeterLocationAtmosphere'] = new_df['MeterLocationAtmosphere'].fillna('METERROOM')
new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].fillna('METALLICBARE')
new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].fillna('METALLICBARE')
new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].fillna('METALLICBARE')
new_df['MainAndBranchService'] = new_df['MainAndBranchService'].fillna('YES')
new_df['NoAccessReasonCdBuilding'] = new_df['NoAccessReasonCdBuilding'].fillna('OTHER')
new_df['ServiceUnderStoop'] = new_df['ServiceUnderStoop'].fillna('YES')
new_df['AccessibleCustomer'] = new_df['AccessibleCustomer'].fillna('YES')
new_df['NoAccessReasonCdCustomer'] = new_df['NoAccessReasonCdCustomer'].fillna('DIDNOTATTEMPT')
new_df['CorrosionSeverityCdCustomer'] = new_df['CorrosionSeverityCdCustomer'].fillna('NONE')
new_df['CorrosionLocationsCdCustomer'] = new_df['CorrosionLocationsCdCustomer'].fillna('NONE')
new_df['CtagEquipmentCd'] = new_df['CtagEquipmentCd'].fillna('OES')
new_df['HeadOfServiceValveNotAccessibleHeight'] = new_df['HeadOfServiceValveNotAccessibleHeight'].fillna('NO')
new_df['GasTurnedOff'] = new_df['GasTurnedOff'].fillna('NO')
new_df['AllServicePipingInspected'] = new_df['AllServicePipingInspected'].fillna('YES')
new_df['CorrosionFoundOnCdCustomer'] = new_df['CorrosionFoundOnCdCustomer'].fillna('OTHER')
new_df['ImproperConditionsFoundCustomer'] = new_df['ImproperConditionsFoundCustomer'].fillna('NO')
new_df['DamagedGasMeterCustomerCd'] = new_df['DamagedGasMeterCustomerCd'].fillna('OTHER')
new_df['InappropriateFlexibleConnectorCds'] = new_df['InappropriateFlexibleConnectorCds'].fillna('THROUGH_WALL')
new_df['AccessibleConEd'] = new_df['AccessibleConEd'].fillna('YES')
new_df['NoAccessReasonCdConed'] = new_df['NoAccessReasonCdConed'].fillna('DIDNOTATTEMPT')
new_df['POESleeveLocationCd'] = new_df['POESleeveLocationCd'].fillna('OTHER')
new_df['InspDoneOnAllSvcsEnteringBldg'] = new_df['InspDoneOnAllSvcsEnteringBldg'].fillna('YES')
new_df['ImproperConditionsFoundConEd'] = new_df['ImproperConditionsFoundConEd'].fillna('NO')
new_df['ImproperSubstandardConditionConEdCds'] = new_df['ImproperSubstandardConditionConEdCds'].fillna('OTHER')
new_df['DamagedGasMeterConEdCd'] = new_df['DamagedGasMeterConEdCd'].fillna('OTHER')
new_df['UnauthorizedMetersFound'] = new_df['UnauthorizedMetersFound'].fillna('NO')
new_df['SupervisorNotified'] = new_df['SupervisorNotified'].fillna('YES')
new_df['PlasticExposed'] = new_df['PlasticExposed'].fillna('NO')
new_df['PlasticExtendsIntoStructure24Inches'] = new_df['PlasticExtendsIntoStructure24Inches'].fillna('YES')
new_df['CorrosionSeverityCdConEd'] = new_df['CorrosionSeverityCdConEd'].fillna('NONE')
new_df['CorrosionFoundOnCdConEd'] = new_df['CorrosionFoundOnCdConEd'].fillna('OTHER')
new_df['CorrosionLocationsCdConEd'] = new_df['CorrosionLocationsCdConEd'].fillna('OTHER')
new_df['NGDRemovedReasonCd'] = new_df['NGDRemovedReasonCd'].fillna('OTHER')
new_df['PoeAccessible'] = new_df['PoeAccessible'].fillna('YES')
new_df['PoeNoAccessReasonCd'] = new_df['PoeNoAccessReasonCd'].fillna('OTHER')
new_df['PoeFoundSealedCondition'] = new_df['PoeFoundSealedCondition'].fillna('NO')
new_df['PoeLeftSealedCondition'] = new_df['PoeLeftSealedCondition'].fillna('NO')
new_df['PoeSealTypeCd'] = new_df['PoeSealTypeCd'].fillna('OTHER')
new_df['GasOdorPresent'] = new_df['GasOdorPresent'].fillna('NO')
new_df['GasLeakCorrosionRelated'] = new_df['GasLeakCorrosionRelated'].fillna('NO')
new_df['LeakItemCd'] = new_df['LeakItemCd'].fillna('OTHER')

In [358]:
value_mapping = {
    'OutsideSet':{
    'METERSET INSIDE':'NO',
    },
    
    'TypeOfDwellingCd':{
        'SINGLE UNIT DWELLING':'SINGLE',
        'COMMERCIAL':'COMMERCIAL',
        'MULTIUNIT': 'MULTIUNIT',
        'MULTI-UNIT RESIDENCE/DWEL':'MULTIUNIT',
        'INDUSTRIAL':'INDUSTRIAL',
        'SINGLE':'SINGLE',
    },
    'MeterLocation':{
        'BASEMENT':'BASEMENT',
        'METER ROOM':'METERROOM',
        'meterroom':'METERROOM',
        # 'METER AT/NEAR POE':'',
        # 'METERPOE':'',
        # 'METERSREMOTE':'',
        # 'ROOMSET':'',
    },
    'MeterLocationAtmosphere':{
        # 'METER AT/NEAR POE;':'METERPOE',
        # 'BELOW GRADE;':'BELOWGRADE',
        # 'AIR CONDITIONING SPACE;':'ACSPACE',
        # 'CLOSED TO THE OUTSIDE;':'CLOSED',
        # 'HEATED SPACE;':'HEATSPACE',
        # 'ROOM SET;':'ROOMSET',
        # 'VENTED TO THE OUTSIDE;':'VENTED',
        # 'METER ROOM;':'METERROOM',
        'BELOW GRADE':'BELOWGRADE',
        'HEATED SPACE':'HEATSPACE',
        'CLOSED TO THE OUTSIDE':'CLOSED',
        'METER AT/NEAR POE':'METERPOE',
        'ROOM SET':'ROOMSET',
        'VENTED TO THE OUTSIDE':'VENTED',
        'METER ROOM':'METERROOM',
        'AIR CONDITIONING SPACE;':'ACSPACE',
        
    },
    'PipingConstructionMaterialCd':{
        # 'METALLIC BARE':'METALLICBARE',
        # 'METALLIC COATED/PAINTED':'METALLICCOAT',
        'METALLICBARE':'METALLICBARE',
        'METALLICCOAT':'METALLICCOAT',
    },
    'MeterConstructionMaterialCd':{
        'METALLICBARE':'METALLICBARE',
        'METALLICCOAT':'METALLICCOAT',
        'METALLIC BARE':'METALLICBARE',
        'METALLIC COATED/PAINTED':'METALLICCOAT',
    },
    'FittingsConstructionMaterialCd':{
        # 'METALLIC BARE':'METALLICBARE',
        # 'METALLIC COATED/PAINTED':'METALLICCOAT',
        'METALLICBARE':'METALLICBARE',
        'METALLICCOAT':'METALLICCOAT',
    },
    'MainAndBranchService':{
        'YES':'YES',
        'NOT AVAILABLE':'NO',
        'NO':'NO'
    },
    'NoAccessReasonCdBuilding':{
        'ACCESS DENIED COVID':'CORONAVIRUS',
        'ADRESS DOES NOT EXIST':'ADDRESSNOTEXIST',
        'BUILDING QUARANTINED':'QUARANTINED',
        'CATS REMARK':'CATS',
        'CUST FAILED KEEP APT':'CUSTFAILAPPT',
        'CUST NOT AT HOME':'CUSTNOTHOME',
    },
    'NoAccessReasonCdCustomer':{
        'CATS':'CATS',
        'CUSTFAILAPPT':'CUSTFAILAPPT',
        'CUSTNOTHOME':'CUSTNOTHOME',
        'DID NOT ATTEMPT':'DIDNOTATTEMPT',
        'CORONAVIRUS':'CORONAVIRUS',
        'ADDRESSNOTEXIST':'ADDRESSNOTEXIST',
        'QUARANTINED':'QUARANTINED',
    },
    'CorrosionSeverityCdCustomer':{
        '1-NO CORROSION SEVERITY':'NONE',
        '2-LOW CORROSION SEVERITY':'LOW',
        '3-MEDIUM CORROSION SEVERITY':'MEDIUM',
    },
    
    'CorrosionLocationsCdCustomer':{
        'BASEMENT;':'BASEMENT',
        'NONE;':'NONE',
        'APARTMENT ;':'APARTMENT',
        'GARAGE;':'GARAGE',
        'HALLWAY;':'HALLWAY',
        'UNDER STAIRS;':'UNDER_STAIRS',
        'VAULT;':'VAULT',
        'NONE;':'NONE',
    },
          
        
    'CtagEquipmentCd':{
        'Other':'O',
        'EXTENSION OF SERVICE':'OES',
    },
    'CorrosionFoundOnCdCustomer':{
        'HOUSE PIPE':'HOUSE_PIPE',
        'CUSTOMER PIPE':'OTHER',
        'GAS CARRIER PIPE':'OTHER',
    },
    'DamagedGasMeterCustomerCd':{
        'BENT GAS PIPING':'BENT_GAS_PIPING',
    },
    'NoAccessReasonCdConed':{
        'Access Denied - COVID':'CORONAVIRUS',
        'Address Does Not Exist':'ADDRESSNOTEXIST',
        'CATS Remark':'CATS',
        'Customer failed to keep appt ':'CUSTFAILAPPT',
        'Customer Not Home':'CUSTNOTHOME',
        'did not Attempt':'DIDNOTATTEMPT',
    },
    'POESleeveLocationCd':{
        'UNDER STAIRS':'UNDER_STAIRS',
    },
    
    'GasOdorPresent':{
        'Yes (Call GERC)':'YES',
    },
        
    'ImproperSubstandardConditionConEdCds':{
        'Improper inside meter location': 'INSIDE_MTR_LOC_IMPR',
        'Inaccessible meter sets':'OTHER',
        'Inappropriate or substandard parts or materials':'PART_MATERIAL',
        ' Jurisdictional pipe not installed to utility specifications':'JURISDICTION_PIPING',
        'Meter set(s) without sufficient support':'OTHER',
        'Meter sets in a hot or humid environment':'OTHER',
        'Meter sets in an environment where they are in danger of damage from falling items around them':'DAMAGE_GAS_METER',
        'Meter sets without access to shut-off valves':'NO_SHUTOFF_VALVE',
        ' No regulator vent terminus':'REGULATOR_VENT_GAS',
        'Structures over a gas facility':'GAS_FCLTY_DEPTH_INCO',
    },
    'DamagedGasMeterConEdCd':{
        'BENT_GAS_PIPING':'BENT_GAS_PIPING',
    },
    'CorrosionSeverityCdConEd':{
        '1-NO CORROSION SEVERITY':'NONE',
        '2-LOW CORROSION SEVERITY':'LOW',
        '3-MEDIUM CORROSION SEVERITY':'MEDIUM',
    },
    'CorrosionFoundOnCdConEd':{
        'EXTENSION':'OTHER',
        'HOUSE PIPE':'OTHER',
        'SERVICE HEAD ADAPTER':'SERVICE_HEAD_ADAPTER',
    },
    'CorrosionLocationsCdConEd':{
        'APARTMENT ;':'APARTMENT',
        'BASEMENT;':'BASEMENT',
        'GARAGE;':'GARAGE',
        'HALLWAY;':'HALLWAY',
        'NONE;':'NONE',
        'UNDER STAIRS;':'UNDER_STAIRS',
        'VAULT;':'VAULT',
    },
    'PoeNoAccessReasonCd':{
        'ACCESS NOT REQUIRED':'ACCESS_NOT_REQUIRED',
        'AOC CONDITION':'OTHER',
        'CUST REFUSED ACCESS':'CUST_REFUSED_ACCESS',
        'NO ACCESS':'NO_ACCESS',
    },
    'LeakItemCd':{
        'METER;':'METER',
        'VALVE;':'VALVE',
        'REGULATOR;METER;':'REGULATOR',
        'FITTING THREADS;':'FITTINGTHREAD',
        'FITTING (ELBOW, TEE, ETC.);':'FITTING',
    }
}

TypeOfDwellingCd_map = value_mapping.get('TypeOfDwellingCd',{})
#working - new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].apply(lambda x:TypeOfDwellingCd_map.get(x,'OTHER') if x and pd.notna(x) else 'SINGLE')
new_df['TypeOfDwellingCd'] = new_df['TypeOfDwellingCd'].apply(lambda x:'SINGLE' if 'single' in x.lower() else TypeOfDwellingCd_map.get(x,'OTHER') if x and pd.notna(x) else 'SINGLE')

MeterLocation_map = value_mapping.get('MeterLocation',{})
new_df['MeterLocation'] = new_df['MeterLocation'].apply(lambda x:'BASEMENT' if 'basement' in x.lower() else 'HALLWAY' if 'hallway' in x.lower() else 'GARAGE' if 'garage' in x.lower() else MeterLocation_map.get(x,'OTHER') if x and pd.notna(x) else 'BASEMENT')

PipingConstructionMaterialCd_map = value_mapping.get('PipingConstructionMaterialCd',{})
new_df['PipingConstructionMaterialCd'] = new_df['PipingConstructionMaterialCd'].apply(lambda x: PipingConstructionMaterialCd_map.get(x,'OTHER') if x and pd.notna(x) else 'METALLICBARE')

MeterConstructionMaterialCd_map = value_mapping.get('MeterConstructionMaterialCd',{})
new_df['MeterConstructionMaterialCd'] = new_df['MeterConstructionMaterialCd'].apply(lambda x: MeterConstructionMaterialCd_map.get(x,'OTHER') if x and pd.notna(x) else 'METALLICBARE')

FittingsConstructionMaterialCd_map = value_mapping.get('FittingsConstructionMaterialCd',{})
new_df['FittingsConstructionMaterialCd'] = new_df['FittingsConstructionMaterialCd'].apply(lambda x: FittingsConstructionMaterialCd_map.get(x,'OTHER') if x and pd.notna(x) else 'METALLICBARE')

MainAndBranchService_map = value_mapping.get('MainAndBranchService',{})
new_df['MainAndBranchService'] = new_df['MainAndBranchService'].apply(lambda x: MainAndBranchService_map.get(x,'OTHER') if x and pd.notna(x) else 'YES')

MainAndBranchService_map = value_mapping.get('MainAndBranchService',{})
new_df['MainAndBranchService'] = new_df['MainAndBranchService'].apply(lambda x: MainAndBranchService_map.get(x,'OTHER') if x and pd.notna(x) else 'YES')

for column, mapping in value_mapping.items():
    if column!='TypeOfDwellingCd' or column!='MeterLocation' or column!='PipingConstructionMaterialCd' or column!='MeterConstructionMaterialCd' or column!='MainAndBranchService' or column!='FittingsConstructionMaterialCd':
        new_df[column] = new_df[column].replace(mapping)
    
new_df = new_df[first_df['OutsideSet']!='YES']

new_df = new_df[first_df['BuildingAccessible']!='NO']

#new_df = new_df.dropna(subset = ['BuildingAccessible'])
    
new_df.to_excel('new df trial39.xlsx', index=False)

C:\Users\ARORASE\AppData\Local\Temp\ipykernel_17912\4221511449.py:205: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_df = new_df[first_df['BuildingAccessible']!='NO']


In [106]:
print(new_df['TypeOfDwellingCd'].unique())
print(TypeOfDwellingCd_map.keys())

['COMMERCIAL' 'MULTIUNIT' 'SINGLE' 'INDUSTRIAL' 'OTHER']
dict_keys(['SINGLE UNIT DWELLING', 'COMMERCIAL', 'MULTIUNIT', 'MULTI-UNIT RESIDENCE/DWEL', 'INDUSTRIAL', 'SINGLE'])
